## Import necessary libraries

In [24]:
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
from sklearn.metrics import f1_score

sys.path.append('../../src')
from rnn.rnn_from_scratch import SimpleRNNModel
from utils.data_preprocessing import TextPreprocessor
from rnn.model_training import create_keras_rnn

import warnings
warnings.filterwarnings("ignore")


In [14]:
train = pd.read_csv('../../datasets/train.csv', index_col='id')
valid = pd.read_csv('../../datasets/valid.csv', index_col='id')
test = pd.read_csv('../../datasets/test.csv', index_col='id')
train.head()

,text,label
id,,
219,Nikmati cicilan 0% hingga 12 bulan untuk pemes...,neutral
209,Kue-kue yang disajikan bikin saya bernostalgia...,positive
436,Ibu pernah bekerja di grab indonesia,neutral
394,Paling suka banget makan siang di sini ayam sa...,positive
592,Pelayanan bus DAMRI sangat baik,positive


In [15]:
test.head()

,text,label
id,,
411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive
729,"Iya benar, dia sedang jaga warung.",neutral
373,Kangkungnya lumayan tapi kepiting saus padangn...,negative
262,Bertempat di braga city walk yang satu gedung ...,positive
177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 219 to 719
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    500 non-null    object
 1   label   500 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 411 to 768
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
 1   label   400 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB


## Preprocessing

In [18]:
preprocessor = TextPreprocessor(vocab_size=10000, max_length=100)
processed_data = preprocessor.preprocess_dataset(train, valid, test, use_vectorizer=False)

train_seq = processed_data['train_sequences']
val_seq = processed_data['val_sequences']
test_seq = processed_data['test_sequences']
train_labels = processed_data['train_labels']
val_labels = processed_data['val_labels']
test_labels = processed_data['test_labels']

vocab_size = processed_data['vocab_size']
num_classes = processed_data['num_classes']

print(f"Vocab size: {vocab_size}")
print(f"Num classes: {num_classes}")
print(f"Sequence shape: {train_seq.shape}")
print(f"Labels distribution: {np.bincount(train_labels)}")

Vocab size: 2796
Num classes: 3
Sequence shape: (500, 100)
Labels distribution: [192 119 189]


## Modelling

In [19]:
configs = {
    # Pengaruh jumlah layer (3 variasi)
    'rnn_1layer': {'hidden_sizes': [64], 'bidirectional': False},
    'rnn_2layer': {'hidden_sizes': [64, 32], 'bidirectional': False},
    'rnn_3layer': {'hidden_sizes': [64, 32, 16], 'bidirectional': False},
    
    # Pengaruh banyak cell (3 variasi)
    'rnn_cells_32': {'hidden_sizes': [32, 32], 'bidirectional': False},
    'rnn_cells_64': {'hidden_sizes': [64, 64], 'bidirectional': False},
    'rnn_cells_128': {'hidden_sizes': [128, 128], 'bidirectional': False},
    
    # Pengaruh arah (2 variasi)
    'rnn_unidirectional': {'hidden_sizes': [64, 32], 'bidirectional': False},
    'rnn_bidirectional': {'hidden_sizes': [64, 32], 'bidirectional': True}
}

In [20]:
training_results = {}

for name, config in configs.items():
    print(f"\nTraining {name}...")
    
    # Create and train
    model = create_keras_rnn(vocab_size, num_classes, config)
    
    history = model.fit(
        train_seq, train_labels,
        validation_data=(val_seq, val_labels),
        epochs=10, batch_size=32, verbose=1
    )
    
    # Evaluate
    y_pred = np.argmax(model.predict(test_seq), axis=1)
    macro_f1 = f1_score(test_labels, y_pred, average='macro')
    
    # Save
    model.save(f'../../models/{name}.h5')
    
    training_results[name] = {
        'config': config,
        'macro_f1': macro_f1,
        'history': history.history
    }
    
    print(f"✅ {name} - F1: {macro_f1:.4f}")


Training rnn_1layer...
Epoch 1/10


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.3532 - loss: 1.0941 - val_accuracy: 0.3600 - val_loss: 1.0497
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5365 - loss: 0.9449 - val_accuracy: 0.4000 - val_loss: 1.1578
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6856 - loss: 0.7450 - val_accuracy: 0.4300 - val_loss: 1.2006
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7815 - loss: 0.5694 - val_accuracy: 0.4100 - val_loss: 1.3479
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9095 - loss: 0.4043 - val_accuracy: 0.3700 - val_loss: 1.6533
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9595 - loss: 0.2445 - val_accuracy: 0.4100 - val_loss: 1.8359
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9924 - loss: 0.1106 - val_accuracy: 0.4100 - val_loss: 1.9329
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9954 - loss: 0.0733 - val_accuracy: 0.4100 - val_loss: 2.

✅ rnn_1layer - F1: 0.3667

Training rnn_2layer...
Epoch 1/10


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - accuracy: 0.3830 - loss: 1.1353 - val_accuracy: 0.3800 - val_loss: 1.0653
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.4918 - loss: 1.0031 - val_accuracy: 0.3000 - val_loss: 1.2068
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.6652 - loss: 0.7739 - val_accuracy: 0.3900 - val_loss: 1.1691
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9216 - loss: 0.3987 - val_accuracy: 0.3900 - val_loss: 1.3482
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.9605 - loss: 0.2042 - val_accuracy: 0.4200 - val_loss: 1.5718
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9949 - loss: 0.0911 - val_accuracy: 0.3800 - val_loss: 1.7633
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 1.0000 - loss: 0.0350 - val_accuracy: 0.4300 - val_loss: 1.7715
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 1.0000 - loss: 0.0218 - val_accuracy: 0.4600 - val_loss: 1

✅ rnn_unidirectional - F1: 0.4401

Training rnn_bidirectional...


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 157ms/step - accuracy: 0.3309 - loss: 1.1755 - val_accuracy: 0.4900 - val_loss: 1.0307
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.6139 - loss: 0.8771 - val_accuracy: 0.5100 - val_loss: 0.9976
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.9141 - loss: 0.4839 - val_accuracy: 0.5300 - val_loss: 1.0226
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.9857 - loss: 0.2068 - val_accuracy: 0.5100 - val_loss: 1.0840
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 1.0000 - loss: 0.0777 - val_accuracy: 0.5300 - val_loss: 1.1096
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 1.0000 - loss: 0.0389 - val_accuracy: 0.5700 - val_loss: 1.1928
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 1.0000 - loss: 0.0230 - val_accuracy: 0.5700 - val_loss: 1.2194
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 1.0000 - loss: 0.0154 - val_accuracy: 0.6000 -

✅ rnn_bidirectional - F1: 0.4998


## Compare models

In [22]:
def compare_implementations(model_path, config, test_data):
    """Compare Keras vs from scratch"""
    test_X, test_y = test_data
    
    # Load Keras model
    keras_model = tf.keras.models.load_model(model_path)
    
    # Create scratch model
    scratch_model = SimpleRNNModel(
        vocab_size=vocab_size,
        embedding_dim=128,
        hidden_sizes=config['hidden_sizes'],
        num_classes=num_classes,
        bidirectional=config['bidirectional']
    )
    
    # Load weights to scratch model
    scratch_model.load_keras_weights(model_path)
    
    # Get predictions
    keras_proba = keras_model.predict(test_X, verbose=0)
    scratch_proba = scratch_model.predict_proba(test_X)
    
    keras_pred = np.argmax(keras_proba, axis=1)
    scratch_pred = scratch_model.predict(test_X)
    
    # Calculate metrics
    keras_f1 = f1_score(test_y, keras_pred, average='macro')
    scratch_f1 = f1_score(test_y, scratch_pred, average='macro')
    
    max_diff = np.max(np.abs(keras_proba - scratch_proba))
    agreement = np.mean(keras_pred == scratch_pred)
    
    
    return {
        'keras_f1': keras_f1,
        'scratch_f1': scratch_f1,
        'max_diff': max_diff,
        'agreement': agreement
    }

In [25]:
test_data = (test_seq, test_labels)
comparison_results = {}

for name, config in configs.items():
    print(f"\nComparing {name}...")
    model_path = f'../../models/{name}.h5'
    
    result = compare_implementations(model_path, config, test_data)
    comparison_results[name] = result
    
    print(f"  Keras F1: {result['keras_f1']:.4f}")
    print(f"  Scratch F1: {result['scratch_f1']:.4f}")
    print(f"  Max diff: {result['max_diff']:.6f}")
    print(f"  Agreement: {result['agreement']:.4f}")


Comparing rnn_1layer...
Weights loaded from ../../models/rnn_1layer.h5


  Keras F1: 0.3667
  Scratch F1: 0.3667
  Max diff: 0.000004
  Agreement: 1.0000

Comparing rnn_2layer...
Weights loaded from ../../models/rnn_2layer.h5


  Keras F1: 0.3811
  Scratch F1: 0.3811
  Max diff: 0.000005
  Agreement: 1.0000

Comparing rnn_3layer...


Weights loaded from ../../models/rnn_3layer.h5


  Keras F1: 0.1844
  Scratch F1: 0.1844
  Max diff: 0.000000
  Agreement: 1.0000

Comparing rnn_cells_32...
Weights loaded from ../../models/rnn_cells_32.h5


...

Weights loaded from ../../models/rnn_cells_128.h5


  Keras F1: 0.2275
  Scratch F1: 0.2275
  Max diff: 0.000000
  Agreement: 1.0000

Comparing rnn_unidirectional...


Weights loaded from ../../models/rnn_unidirectional.h5


  Keras F1: 0.4401
  Scratch F1: 0.4401
  Max diff: 0.000002
  Agreement: 1.0000

Comparing rnn_bidirectional...


Weights loaded from ../../models/rnn_bidirectional.h5
  Keras F1: 0.4998
  Scratch F1: 0.2343
  Max diff: 0.781861
  Agreement: 0.4075
